# Hugging Face Inference Providers

 > Disclaimer: This post has been translated to English using a machine translation model. Please, let me know if you find any mistakes.

It is clear that the largest hub of AI models is Hugging Face. And now they are offering the possibility to perform inference on some of their models using serverless GPU providers.

One of those models is [Wan-AI/Wan2.1-T2V-14B](https://huggingface.co/Wan-AI/Wan2.1-T2V-14B), which as of writing this post is the best open-source video generation model, as can be seen in the [Artificial Analysis Video Generation Arena Leaderboard](https://artificialanalysis.ai/text-to-video/arena?tab=Leaderboard)
![video generation arena leaderboard](https://images.maximofn.com/Video-arena-leaderboard-wan21.webp)

If we look at its [modelcard](https://huggingface.co/Wan-AI/Wan2.1-T2V-14B) we can see on the right a button that says `Replicate`.
![Wan2.1-T2V-14B modelcard](https://images.maximofn.com/Wan2.1-T2V-14B.webp)

## Inference providers

If we go to the [Inference providers](https://huggingface.co/settings/inference-providers) settings page, we will see something like this
![Inference Providers](https://images.maximofn.com/Inference%20Providers.webp)
Where we can press the button with a key to enter the API KEY of the provider we want to use, or leave selected the path with two dots. If we choose the first option, it will be the provider who charges us for the inference, while in the second option, it will be Hugging Face who charges us for the inference. So do what suits you best.

## Inference with Replicate

In my case, I obtained an API KEY from Replicate and added it to a file called `.env`, which is where I will store the API KEYS and which you should not upload to GitHub, GitLab, or your project repository.
The `.env` must have this format
``` python
HUGGINGFACE_TOKEN_INFERENCE_PROVIDERS="hf_aL...AY"REPLICATE_API_KEY="r8_Sh...UD"```

Where `HUGGINGFACE_TOKEN_INFERENCE_PROVIDERS` is a token you need to obtain from [Hugging Face](https://huggingface.co/settings/tokens) and `REPLICATE_API_KEY` is the API KEY of Replicate which you can obtain from [Replicate](https://replicate.com/account/api-tokens).

### Reading the API Keys

The first thing we need to do is read the API KEYS from the `.env` file

In [1]:
import os
import dotenv
dotenv.load_dotenv()

REPLICATE_API_KEY = os.getenv("REPLICATE_API_KEY")
HUGGINGFACE_TOKEN_INFERENCE_PROVIDERS = os.getenv("HUGGINGFACE_TOKEN_INFERENCE_PROVIDERS")

### Logging in the Hugging Face hub

To be able to use the Wan-AI/Wan2.1-T2V-14B model, as it is on the Hugging Face hub, we need to log in.

In [2]:
from huggingface_hub import login
login(HUGGINGFACE_TOKEN_INFERENCE_PROVIDERS)

### Inference Client

Now we create an inference client, we have to specify the provider, the API KEY and in this case, additionally, we are going to set a `timeout` of 1000 seconds, because by default it is 60 seconds and the model takes quite a while to generate the video.

In [3]:
from huggingface_hub import InferenceClient

client = InferenceClient(
	provider="replicate",
	api_key=REPLICATE_API_KEY,
	timeout=1000
)

### Video Generation

We already have everything to generate our video. We use the `text_to_video` method of the client, pass it the prompt, and tell it which model from the hub we want to use; if not, it will use the default one.

In [6]:
video = client.text_to_video(
	"Funky dancer, dancing in a rehearsal room. She wears long hair that moves to the rhythm of her dance.",
	model="Wan-AI/Wan2.1-T2V-14B",
)

### Saving the video

Finally, we save the video, which is of type `bytes`, to a file on our disk.

In [7]:
output_path = "output_video.mp4"
with open(output_path, "wb") as f:
    f.write(video)
print(f"Video saved to: {output_path}")

Video saved to: output_video.mp4


## Generated video

This is the video generated by the model
<video src="https://images.maximofn.com/wan2_1_video.webm" controls></video>